In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff 
from plotly.grid_objs import Grid, Column 
import colorlover as cl

#####################################################
#####################################################

                ##### STOCK X #####
                     # DATA #
                #### CHALLENGE ####
            
#####################################################
#####################################################

path = 'https://s3.amazonaws.com/stockx-sneaker-analysis/wp-content/uploads/2019/02/StockX-Data-Contest-2019.xlsx'

df = pd.read_excel(path, sheet_name=1)

df.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,2016-11-23,11.0,California
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,2016-11-23,11.5,Kentucky
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,2017-02-11,11.0,Rhode Island


In [2]:
off_white = df.copy()
off_white = off_white[off_white['Brand'].isin(['Off-White'])]

yeezy = df.copy()
yeezy = yeezy[yeezy['Brand'].isin([' Yeezy'])]

print('Shape of Off-White dataframe : {}'.format(off_white.shape))
print('Shape of Yeezy dataframe : {}'.format(yeezy.shape))

Shape of Off-White dataframe : (27794, 8)
Shape of Yeezy dataframe : (72162, 8)


In [3]:
#gonna do some simple subtraction!
def profit_after_fees(x,y):
    '''
    Taking into account StockX's seller fees at 9.5% maximum and the additional 3% transaction fees.
    Should be 12.5% max 
    https://help.stockx.com/selling-on-stockx/what-are-stockx-selling-fees
    '''
    profit = x - y
    return round((profit * .875),2)

In [4]:
yeezy['Profit After Fees'] = yeezy.apply(lambda x: profit_after_fees(x['Sale Price'],x['Retail Price']), axis=1)
yeezy.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region,Profit After Fees
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California,767.38
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California,406.88
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,2016-11-23,11.0,California,411.25
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,2016-11-23,11.5,Kentucky,748.12
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,2017-02-11,11.0,Rhode Island,532.00


In [5]:
yeezy['Favorite'] = yeezy.groupby(['Buyer Region','Sneaker Name'])['Sneaker Name'].transform('count')
yeezy.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region,Profit After Fees,Favorite
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California,767.38,115
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California,406.88,58
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,2016-11-23,11.0,California,411.25,58
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,2016-11-23,11.5,Kentucky,748.12,8
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,2017-02-11,11.0,Rhode Island,532.00,3


In [6]:
z = yeezy.groupby(['Buyer Region','Sneaker Name'], as_index=False)['Favorite'].max()
z.head()

,Buyer Region,Sneaker Name,Favorite
0,Alabama,Adidas-Yeezy-Boost-350-Low-Pirate-Black-2016,1
1,Alabama,Adidas-Yeezy-Boost-350-Low-V2-Beluga,4
2,Alabama,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,47
3,Alabama,Adidas-Yeezy-Boost-350-V2-Blue-Tint,47
4,Alabama,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1


In [7]:
xz = z.pivot('Buyer Region','Sneaker Name','Favorite').fillna(0)
xz.head()

Sneaker Name,Adidas-Yeezy-Boost-350-Low-Moonrock,Adidas-Yeezy-Boost-350-Low-Oxford-Tan,Adidas-Yeezy-Boost-350-Low-Pirate-Black-2015,Adidas-Yeezy-Boost-350-Low-Pirate-Black-2016,Adidas-Yeezy-Boost-350-Low-Turtledove,Adidas-Yeezy-Boost-350-Low-V2-Beluga,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,Adidas-Yeezy-Boost-350-V2-Blue-Tint,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,Adidas-Yeezy-Boost-350-V2-Core-Black-White,Adidas-Yeezy-Boost-350-V2-Cream-White,Adidas-Yeezy-Boost-350-V2-Semi-Frozen-Yellow,Adidas-Yeezy-Boost-350-V2-Sesame,Adidas-Yeezy-Boost-350-V2-Zebra,adidas-Yeezy-Boost-350-V2-Butter,adidas-Yeezy-Boost-350-V2-Static,adidas-Yeezy-Boost-350-V2-Static-Reflective
Buyer Region,,,,,,,,,,,,,,,,,,,,
Alabama,0.0,0.0,0.0,1.0,0.0,4.0,47.0,47.0,0.0,0.0,1.0,1.0,5.0,51.0,36.0,24.0,71.0,48.0,23.0,16.0
Alaska,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,1.0,1.0,0.0,9.0,5.0,3.0,4.0,4.0,5.0,0.0
Arizona,1.0,0.0,1.0,1.0,1.0,5.0,147.0,139.0,1.0,2.0,2.0,10.0,6.0,152.0,62.0,81.0,142.0,164.0,59.0,29.0
Arkansas,1.0,0.0,0.0,0.0,0.0,1.0,20.0,18.0,1.0,0.0,0.0,2.0,1.0,26.0,7.0,10.0,12.0,26.0,13.0,3.0
California,25.0,16.0,28.0,38.0,22.0,115.0,2146.0,1682.0,58.0,58.0,82.0,250.0,142.0,1653.0,587.0,1045.0,1737.0,1810.0,923.0,696.0


In [8]:
xz['Favorite Sneaker'] = xz[['Adidas-Yeezy-Boost-350-Low-Moonrock',
       'Adidas-Yeezy-Boost-350-Low-Oxford-Tan',
       'Adidas-Yeezy-Boost-350-Low-Pirate-Black-2015',
       'Adidas-Yeezy-Boost-350-Low-Pirate-Black-2016',
       'Adidas-Yeezy-Boost-350-Low-Turtledove',
       'Adidas-Yeezy-Boost-350-Low-V2-Beluga',
       'Adidas-Yeezy-Boost-350-V2-Beluga-2pt0',
       'Adidas-Yeezy-Boost-350-V2-Blue-Tint',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-Copper',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-Green',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-Red',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-White',
       'Adidas-Yeezy-Boost-350-V2-Cream-White',
       'Adidas-Yeezy-Boost-350-V2-Semi-Frozen-Yellow',
       'Adidas-Yeezy-Boost-350-V2-Sesame', 'Adidas-Yeezy-Boost-350-V2-Zebra',
       'adidas-Yeezy-Boost-350-V2-Butter', 'adidas-Yeezy-Boost-350-V2-Static',
       'adidas-Yeezy-Boost-350-V2-Static-Reflective']].max(axis=1)

In [11]:
l = []
for i in range(0,len(xz)):
    p = (xz == xz['Favorite Sneaker'][i]).idxmax(axis=1)[i]
    l.append(p)

new_df = pd.DataFrame(l,columns={'Sneaker'})
new_df.head()
new_df['State'] = xz.index
new_df.head()

,Sneaker,State
0,Adidas-Yeezy-Boost-350-V2-Zebra,Alabama
1,Adidas-Yeezy-Boost-350-V2-Cream-White,Alaska
2,adidas-Yeezy-Boost-350-V2-Butter,Arizona
3,Adidas-Yeezy-Boost-350-V2-Cream-White,Arkansas
4,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,California


In [16]:
yeezy_copy = yeezy.copy()
yeezy_copy = yeezy_copy['Buyer Region'].value_counts().reset_index().rename(columns={'index': 'State', 'Buyer Region': 'Total Sales'})

In [18]:
yeezy_df = new_df.copy()
yeezy_df.head()

,Sneaker,State
0,Adidas-Yeezy-Boost-350-V2-Zebra,Alabama
1,Adidas-Yeezy-Boost-350-V2-Cream-White,Alaska
2,adidas-Yeezy-Boost-350-V2-Butter,Arizona
3,Adidas-Yeezy-Boost-350-V2-Cream-White,Arkansas
4,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,California


In [19]:

ids = [
'AL',
'AK',    
'AZ',
'AR',
'CA',
'CO',
'CT',
'DE',
'DC',
'FL',
'GA',
'HI',
'ID',
'IL',
'IN',
'IA',    
'KS',
'KY',
'LA',
'ME',
'MD',
'MA',
'MI',
'MN',
'MS',
'MO',
'MT',
'NE',
'NV',
'NH',
'NJ',
'NM',
'NY',
'NC',
'ND',
'OH',
'OK',
'OR',
'PA',
'RI',
'SC',
'SD',
'TN',
'TX',
'UT',
'VT',
'VA',
'WA',
'WV',
'WI',
'WY']
ids_l = pd.DataFrame(ids,columns={'IDs'})
ids_l.head()

,IDs
0,AL
1,AK
2,AZ
3,AR
4,CA


In [20]:
ids_l['State'] = xz.index
ids_l.head()
yeezy_df = pd.merge(yeezy_df,ids_l, on='State', how='inner', sort=True)
yeezy_df.head()

,Sneaker,State,IDs
0,Adidas-Yeezy-Boost-350-V2-Zebra,Alabama,AL
1,Adidas-Yeezy-Boost-350-V2-Cream-White,Alaska,AK
2,adidas-Yeezy-Boost-350-V2-Butter,Arizona,AZ
3,Adidas-Yeezy-Boost-350-V2-Cream-White,Arkansas,AR
4,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,California,CA


In [21]:
dropped_dc = yeezy_df.copy()
dropped_dc = dropped_dc.set_index('IDs')
dropped_dc = dropped_dc.drop(index='DC')
dropped_dc = dropped_dc.reset_index()
dropped_dc.head()

,IDs,Sneaker,State
0,AL,Adidas-Yeezy-Boost-350-V2-Zebra,Alabama
1,AK,Adidas-Yeezy-Boost-350-V2-Cream-White,Alaska
2,AZ,adidas-Yeezy-Boost-350-V2-Butter,Arizona
3,AR,Adidas-Yeezy-Boost-350-V2-Cream-White,Arkansas
4,CA,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,California


In [22]:
#divide each sales amount by population of State = percapita
#Find yeezys per capita

IDs = dropped_dc['IDs'].values
pops = [
    4887871,
    737438,
    7171646,
    3013825,
    39557045,
    5695564,
    3572665,
    967171,
    21299325,
    10519475,
    1420491,
    1754208,
    12741080,
    6691878,
    3156145,
    2911505,
    4468402,
    4659978,
    1338404,
    6042718,
    6902149,
    9998915,
    5611179,
    2986530,
    6126452,
    1062305,
    1929268,
    3034392,
    1356458,
    9032873,
    2095428,
    19542209,
    10383620,
    760077,
    11689442,
    3943079,
    4190793,
    12807060,
    1057315,
    5084127,
    882235,
    6770010,
    28701845,
    3161105,
    626299,
    8517685,
    7535591,
    1805832,
    5813568,
    577737
]

pops = pd.DataFrame(pops,columns={'Population'})
pops['IDs'] = IDs
pops.head()

,Population,IDs
0,4887871,AL
1,737438,AK
2,7171646,AZ
3,3013825,AR
4,39557045,CA


In [23]:
yeezy_df = pd.merge(yeezy_df, pops, how='inner', on='IDs')
yeezy_df.head()

,Sneaker,State,IDs,Population
0,Adidas-Yeezy-Boost-350-V2-Zebra,Alabama,AL,4887871
1,Adidas-Yeezy-Boost-350-V2-Cream-White,Alaska,AK,737438
2,adidas-Yeezy-Boost-350-V2-Butter,Arizona,AZ,7171646
3,Adidas-Yeezy-Boost-350-V2-Cream-White,Arkansas,AR,3013825
4,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,California,CA,39557045


In [26]:
months = np.datetime_as_string(yeezy['Order Date'].values,unit='M')
months.shape
yeezy['Order Month'] = months
yeezy['Count'] = np.ones(yeezy.shape[0])
monthly_df = yeezy.groupby(['Buyer Region', 'Order Month'], as_index=False).sum().pivot('Buyer Region','Order Month','Count').fillna(0).cumsum(axis=1)
print(monthly_df.shape)

(51, 18)


In [27]:
yeezy_monthly = monthly_df.reset_index().rename(columns={'Buyer Region': 'State'})
yeezy_monthly = pd.merge(yeezy_monthly, ids_l, how='inner', on='State')
yeezy_monthly.head()

,State,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,IDs
0,Alabama,6.0,8.0,25.0,51.0,67.0,78.0,84.0,96.0,104.0,117.0,134.0,156.0,168.0,184.0,241.0,304.0,352.0,375.0,AL
1,Alaska,0.0,0.0,1.0,6.0,10.0,10.0,10.0,10.0,11.0,12.0,16.0,17.0,19.0,21.0,25.0,37.0,39.0,41.0,AK
2,Arizona,3.0,9.0,59.0,139.0,196.0,228.0,266.0,290.0,316.0,357.0,440.0,487.0,518.0,571.0,703.0,870.0,974.0,1005.0,AZ
3,Arkansas,0.0,1.0,6.0,13.0,21.0,23.0,29.0,35.0,38.0,42.0,54.0,61.0,68.0,79.0,95.0,117.0,136.0,141.0,AR
4,California,94.0,206.0,920.0,2266.0,2907.0,3361.0,3762.0,4044.0,4417.0,4946.0,5966.0,6471.0,6954.0,7634.0,9250.0,10955.0,12526.0,13113.0,CA


In [28]:
yeezy_monthly = yeezy_monthly.set_index(['IDs'])
yeezy_monthly = yeezy_monthly.drop('DC')
print(yeezy_monthly.shape)
yeezy_monthly = yeezy_monthly.reset_index()

(50, 19)


In [30]:
yeezy_cap = yeezy.groupby(['Buyer Region', 'Order Month'], as_index=False).sum().pivot('Buyer Region', 'Order Month', 'Count').fillna(0).cumsum(axis=1)
yeezy_cap = yeezy_cap.reset_index().rename(columns={'Buyer Region': 'State'})
yeezy_cap = pd.merge(yeezy_cap, ids_l,how='inner',on='State')
yeezy_cap = yeezy_cap.set_index(['IDs'])
yeezy_cap = yeezy_cap.drop('DC')
yeezy_cap = yeezy_cap.reset_index()
yeezy_cap = pd.merge(yeezy_cap, pops, on='IDs', how='inner')
yeezy_cap.head()

,IDs,State,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,...,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,Population
0,AL,Alabama,6.0,8.0,25.0,51.0,67.0,78.0,84.0,96.0,...,117.0,134.0,156.0,168.0,184.0,241.0,304.0,352.0,375.0,4887871
1,AK,Alaska,0.0,0.0,1.0,6.0,10.0,10.0,10.0,10.0,...,12.0,16.0,17.0,19.0,21.0,25.0,37.0,39.0,41.0,737438
2,AZ,Arizona,3.0,9.0,59.0,139.0,196.0,228.0,266.0,290.0,...,357.0,440.0,487.0,518.0,571.0,703.0,870.0,974.0,1005.0,7171646
3,AR,Arkansas,0.0,1.0,6.0,13.0,21.0,23.0,29.0,35.0,...,42.0,54.0,61.0,68.0,79.0,95.0,117.0,136.0,141.0,3013825
4,CA,California,94.0,206.0,920.0,2266.0,2907.0,3361.0,3762.0,4044.0,...,4946.0,5966.0,6471.0,6954.0,7634.0,9250.0,10955.0,12526.0,13113.0,39557045


In [31]:
months = ['2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
       '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08',
       '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02']

# I want to represent each value as : YEEZYS OWNED PER 100,000 IN STATE
for month in months:
    yeezy_cap[month] = (yeezy_cap[month].divide(yeezy_cap['Population']))
    yeezy_cap[month] = yeezy_cap[month].multiply(100000)
    yeezy_cap[month] = round(yeezy_cap[month],1)
yeezy_cap.head()

,IDs,State,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,...,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,Population
0,AL,Alabama,0.1,0.2,0.5,1.0,1.4,1.6,1.7,2.0,...,2.4,2.7,3.2,3.4,3.8,4.9,6.2,7.2,7.7,4887871
1,AK,Alaska,0.0,0.0,0.1,0.8,1.4,1.4,1.4,1.4,...,1.6,2.2,2.3,2.6,2.8,3.4,5.0,5.3,5.6,737438
2,AZ,Arizona,0.0,0.1,0.8,1.9,2.7,3.2,3.7,4.0,...,5.0,6.1,6.8,7.2,8.0,9.8,12.1,13.6,14.0,7171646
3,AR,Arkansas,0.0,0.0,0.2,0.4,0.7,0.8,1.0,1.2,...,1.4,1.8,2.0,2.3,2.6,3.2,3.9,4.5,4.7,3013825
4,CA,California,0.2,0.5,2.3,5.7,7.3,8.5,9.5,10.2,...,12.5,15.1,16.4,17.6,19.3,23.4,27.7,31.7,33.1,39557045


In [33]:
def convert_colorscale_format(colorscale):
    plotly_colorscale = []
    for index, sec_value in enumerate(np.linspace(0, 1, len(colorscale))):
        plotly_colorscale.append([sec_value, colorscale[index]])
    return plotly_colorscale

x = cl.scales['5']['seq']['YlGn']
x100 = cl.to_hsl(cl.interp(x,100))
from IPython.display import HTML
test = convert_colorscale_format(x100)
HTML(cl.to_html(x100))

In [34]:
yeezy_cap['Population'] = yeezy_cap['Population'].astype(str)

yeezy_cap['text'] = 'Yeezys Resold per 100,000' + '<br>' +\
     yeezy_cap['State'] + '<br>' +\
    'State Population : ' + yeezy_cap['Population'] + '<br>' +\
    'Favorite Sneaker : ' + yeezy_df['Sneaker']

columns = []
cols = yeezy_cap.columns
for col in cols: 
    column_name = '{}'.format(col)
    a_columns = Column(list(yeezy_cap[col]), column_name)
    columns.append(a_columns)

grid3 = Grid(columns)
grid_url = py.grid_ops.upload(grid3,'yeezy_gridfinal_show', auto_open=False)


In [35]:
## THIS IS WHERE THE PLOTLY MAGIC HAPPENS

months = ['2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
       '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08',
       '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02']


layout = dict(
    title='Yeezys Resold Per Capita by State <br> Interactive Choropleth Between <br> (09/2017 - 02/2019) <br> (Hover for Details)',
    font=dict(family='Helvetica',size=15),
    showlegend=True,
    geo = dict(
        scope = 'usa',
        projection = dict(type='albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255,255,255)',
        ),
    updatemenus = [dict(
        type='buttons',
        showactive=False,
        x=0.05,
        y=1,
        xanchor='right',
        yanchor='top',
        pad=dict(t=0,r=10),
        buttons=[dict(
            label='Play',
            method='animate',
            args=[None,
                 dict(fromcurrent=True,
                     frame=dict(
                         duration=500,redraw=True),
                     transition=dict(
                         duration=500, easing='elastic-out')
                     )]
        )]
    )],
)

figure = {
    'data': [],
    'layout': layout,
    'frames': [],   
}

sliders = dict(
        active=0,
        x = 0.1,
        y = 0,
        yanchor = 'top',
        xachor = 'left',
        currentvalue = dict(
            font = dict(
                size=20
            ),
            prefix = 'Year - Month: ',
            xanchor='right',
            visible = True,
        ),
        pad = dict(
            b=10,
            t=50
        ),
        transition = dict(
            duration=300,
            easing = 'elastic-in-out'
        ),
        steps = [],
        args = [dict(
            transition=dict(duration=500, easing='elastic-in-out')
        )],
        initialValue = months[0],
        plotlycommand = 'animate',
        values = months,
        visible=True
        )

# for month in months:
data_dict = {
    'type': 'choropleth',
    'colorscale': test,
    'zsrc': grid3.get_column_reference('{}'.format(months[0])),
    'locationssrc': grid3.get_column_reference('IDs'),
    'textsrc': grid3.get_column_reference('text'),
    'locationmode': 'USA-states',
    'marker': {
        'line': {
            'color': 'rgb(255,255,255)',
            'width': 2
        }
    },
    'colorbar': {
        'title': {
            'text': 'Yeezys Resold Per Capita <br> Per 100,000',
            'font': {
                'size': 20
            }
        },
        'showticksuffix': 'first',
        'ticksuffix': ' Yeezys Resold Per 100,000',
    },
    'name': months[0]}
figure['data'].append(data_dict)

for month in months:
    frame = {
            'data': [], 
            'name': str(month)
    }
#     for month in months:
    data_dict = {
        'type': 'choropleth',
        'colorscale': test,
        'zsrc': grid3.get_column_reference('{}'.format(month)),
        'locationssrc': grid3.get_column_reference('IDs'),
        'textsrc': grid3.get_column_reference('text'),
        'locationmode': 'USA-states',
        'marker': {
            'line': {
                'color': 'rgb(255,255,255)',
                'width': 2
            }
        },
        'colorbar': {
            'title': {
                'text': 'Yeezys Resold Per Capita <br> Per 100,000',
                'font': {
                    'size': 20
            }
        },
            'showticksuffix': 'first',
            'ticksuffix': ' Yeezys Resold Per 100,000',
        },
        'name': month
    }

    frame['data'].append(data_dict)
        
    figure['frames'].append(frame)
    
    slider_step = {'args':[
        [month],
        {'frame': {'duration':300, 'redraw':True},
         'mode': 'immediate',
         'transition': {'duration':300}}
        ],
        'label': month,
        'method':'animate'}
    sliders['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders]

final = py.create_animations(figure,filename='yeezys_final_WOW',auto_open=False)
final

'https://plot.ly/~Ruwai/74/'

In [1]:
#okay so i'm going to create another chart on the comparison of 
#standardized average sales?
#or maybe some simple linear regression to predict sales?
#i'm down
#we've gotta do some train test splits

